In [1]:
# Установка необходимых библиотек
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 191.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.5/599.5 kB 94.2 MB/s eta 0:00:00


In [2]:
# Импорт библиотек
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

# Чтение данных из CSV файла
data = pd.read_csv('real_estate_dataset.csv')

# Предварительная обработка данных

# Проверка наличия пропусков в данных
print("Количество пропусков в каждом столбце:")
print(data.isnull().sum())

# Определяем числовые и категориальные столбцы
numerical_cols = data.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = data.select_dtypes(include=['object']).columns.tolist()

# Заполняем пропуски в числовых столбцах нулями
data[numerical_cols] = data[numerical_cols].fillna(0)

# Заполняем пропуски в категориальных столбцах значением 'other'
data[categorical_cols] = data[categorical_cols].fillna('other')

# Проверяем датасет на дубликаты и удаляем их при наличии
duplicates = data[data.duplicated()]
print(f"Количество дубликатов: {duplicates.shape[0]}")

data = data.drop_duplicates()
print(f"Размер данных после удаления дубликатов: {data.shape}")

# Кодирование категориальных переменных с помощью LabelEncoder
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

# Удаление столбцов с уникальными значениями
unique_cols = [col for col in data.columns if data[col].nunique() == 1]
print(f"Столбцы с уникальными значениями: {unique_cols}")

data = data.drop(columns=unique_cols)

# Масштабирование числовых признаков со средним нулевым значением
scaler = StandardScaler()
scaled_features = scaler.fit_transform(data[numerical_cols])

# Создаем DataFrame с масштабированными признаками
scaled_features_df = pd.DataFrame(scaled_features, columns=numerical_cols)

# Обновляем данные
data[numerical_cols] = scaled_features_df

# Используем только 20% данных для ускорения работы
data = data.sample(frac=0.2, random_state=42)

# Разделение на тренировочную и тестовую выборки
X = data.drop(['ID', 'Price'], axis=1)
y = data['Price']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)


Количество пропусков в каждом столбце:
ID                    0
Square_Feet           0
Num_Bedrooms          0
Num_Bathrooms         0
Num_Floors            0
Year_Built            0
Has_Garden            0
Has_Pool              0
Garage_Size           0
Location_Score        0
Distance_to_Center    0
Price                 0
dtype: int64
Количество дубликатов: 0
Размер данных после удаления дубликатов: (500, 12)
Столбцы с уникальными значениями: []


In [3]:
from sklearn.svm import SVR
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Создание и обучение модели SVR с параметрами по умолчанию
svr_model = SVR()
svr_model.fit(X_train, y_train)

# Предсказания на тренировочной и тестовой выборках
y_train_pred = svr_model.predict(X_train)
y_test_pred = svr_model.predict(X_test)

# Вычисление метрик
def print_metrics(y_true, y_pred, dataset_type="Тренировочная"):
    r2 = r2_score(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    print(f"{dataset_type} выборка:")
    print(f"R2 Score: {r2}")
    print(f"MSE: {mse}")
    print(f"MAE: {mae}\n")

print_metrics(y_train, y_train_pred, "Тренировочная")
print_metrics(y_test, y_test_pred, "Тестовая")

# Вывод о качестве обученной модели
print("Вывод: Модель SVR с параметрами по умолчанию показала определенный уровень качества, однако метрики могут быть улучшены путем настройки гиперпараметров.")


Тренировочная выборка:
R2 Score: 0.976996894712305
MSE: 0.021617369734806845
MAE: 0.11298229740200198

Тестовая выборка:
R2 Score: 0.712388529949898
MSE: 0.3283601310142313
MAE: 0.42349769110390934

Вывод: Модель SVR с параметрами по умолчанию показала определенный уровень качества, однако метрики могут быть улучшены путем настройки гиперпараметров.


In [4]:
import optuna
from sklearn.model_selection import cross_val_score

# Настройка гиперпараметров модели с помощью Optuna
def objective(trial):
    # Гиперпараметры для подбора
    kernel = trial.suggest_categorical('kernel', ['rbf', 'poly'])
    C = trial.suggest_float('C', 1, 10)
    gamma = trial.suggest_float('gamma', 0.01, 1)
    
    # Создание модели с текущими гиперпараметрами
    svr_model = SVR(kernel=kernel, C=C, gamma=gamma)
    
    # Кросс-валидация для оценки модели
    scores = cross_val_score(svr_model, X_train, y_train, cv=2, scoring='neg_mean_squared_error')
    
    # Среднее значение MSE (чем меньше, тем лучше)
    mse = -scores.mean()
    return mse

# Запуск оптимизации гиперпараметров
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)

# Лучшие гиперпараметры
print("Наилучшие гиперпараметры:")
print(study.best_params)

# Обучение модели с найденными лучшими гиперпараметрами
best_params = study.best_params
svr_model_optimized = SVR(kernel=best_params['kernel'], C=best_params['C'], gamma=best_params['gamma'])
svr_model_optimized.fit(X_train, y_train)

# Предсказания на тренировочной и тестовой выборках
y_train_pred_optimized = svr_model_optimized.predict(X_train)
y_test_pred_optimized = svr_model_optimized.predict(X_test)

# Вычисление метрик
print_metrics(y_train, y_train_pred_optimized, "Тренировочная (оптимизированная)")
print_metrics(y_test, y_test_pred_optimized, "Тестовая (оптимизированная)")


[I 2024-12-22 09:41:55,488] A new study created in memory with name: no-name-be7cb2f5-5b12-4bba-a3f1-76fc802ebbcd
[I 2024-12-22 09:41:55,497] Trial 0 finished with value: 0.9339729020330018 and parameters: {'kernel': 'rbf', 'C': 1.857670247290942, 'gamma': 0.9337665399797551}. Best is trial 0 with value: 0.9339729020330018.
[I 2024-12-22 09:41:55,504] Trial 1 finished with value: 0.3737809500188184 and parameters: {'kernel': 'poly', 'C': 6.291600057861883, 'gamma': 0.6183879151246525}. Best is trial 1 with value: 0.3737809500188184.
[I 2024-12-22 09:41:55,511] Trial 2 finished with value: 0.8426702239504515 and parameters: {'kernel': 'rbf', 'C': 1.8122340766437168, 'gamma': 0.4730107443908008}. Best is trial 1 with value: 0.3737809500188184.
[I 2024-12-22 09:41:55,517] Trial 3 finished with value: 0.8845779768545109 and parameters: {'kernel': 'poly', 'C': 3.4819512450964836, 'gamma': 0.02971493342306633}. Best is trial 1 with value: 0.3737809500188184.
[I 2024-12-22 09:41:55,524] Trial

Наилучшие гиперпараметры:
{'kernel': 'poly', 'C': 2.9678100463195642, 'gamma': 0.997229558358445}
Тренировочная (оптимизированная) выборка:
R2 Score: 0.9909530277708205
MSE: 0.008501971417020776
MAE: 0.0884912003840678

Тестовая (оптимизированная) выборка:
R2 Score: 0.9257980967885755
MSE: 0.0847147947742288
MAE: 0.21210683809285338

